In [1]:
import os
from typing import Tuple

import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from lightautoml.dataset.roles import DatetimeRole

from lightautoml.spark.tasks.base import SparkTask
from lightautoml.spark.utils import SparkDataFrame
from lightautoml.spark.automl.presets.tabular_presets import SparkTabularAutoML

/home/user/projects/LightAutoML/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_spark_session():
    if os.environ.get("SCRIPT_ENV", None) == "cluster":
        spark_sess = SparkSession.builder.getOrCreate()
    else:
        spark_sess = (
            SparkSession
            .builder
            .master("local[*]")
            .config("spark.jars", "../../jars/spark-lightautoml_2.12-0.1.jar")
            .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.9.5")
            .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")
            .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
            .config("spark.kryoserializer.buffer.max", "512m")
            .config("spark.cleaner.referenceTracking.cleanCheckpoints", "true")
            .config("spark.cleaner.referenceTracking", "true")
            .config("spark.cleaner.periodicGC.interval", "1min")
            .config("spark.sql.shuffle.partitions", "16")
            .config("spark.driver.memory", "55g")
            .config("spark.executor.memory", "55g")
            .config("spark.sql.execution.arrow.pyspark.enabled", "true")
            .getOrCreate()
        )

    spark_sess.sparkContext.setCheckpointDir("/tmp/spark_checkpoints")

    spark_sess.sparkContext.setLogLevel("OFF")

    return spark_sess

In [3]:
spark = get_spark_session()

https://mmlspark.azureedge.net/maven added as a remote repository with the name: repo-1


:: loading settings :: url = jar:file:/home/user/projects/LightAutoML/.venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/user/.ivy2/cache
The jars for the packages stored in: /home/user/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5478a09a-ac42-4b21-b6cc-8b40cc50b00f;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;0.9.5 in central
	found com.microsoft.azure#synapseml-core_2.12;0.9.5 in central
	found org.scalactic#scalactic_2.12;3.0.5 in central
	found org.scala-lang#scala-reflect;2.12.4 in central
	found io.spray#spray-json_2.12;1.3.2 in central
	found com.jcraft#jsch;0.1.54 in user-list
	found org.apache.httpcomponents#httpclient;4.5.6 in user-list
	found org.apache.httpcomponents#httpcore;4.4.10 in user-list
	found commons-logging#commons-logging;1.2 in user-list
	found commons-codec#commons-codec;1.10 in user-list
	found org.apache.httpcomponents#httpmime;4.5.6 in user-list
	found com.linkedin.isolation-forest#isolation-forest_3.2.0_2.12;2.0.8 in central
	found com.

22/05/24 11:41:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/24 11:41:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Dataset reading

(reads only 1M rows)

In [4]:
data = spark. \
        read. \
        csv("file:///opt/spark_data/HIGGS.csv", header=False, escape="\""). \
        limit(1_000_000)

In [7]:
data = data.cache()
data.write.mode('overwrite').format('noop').save()

In [5]:
data.count()

1000000

In [6]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

## Data Set Information:

The data has been produced using Monte Carlo simulations. The first 21 features (columns 2-22) are kinematic properties measured by the particle detectors in the accelerator. The last seven features are functions of the first 21 features; these are high-level features derived by physicists to help discriminate between the two classes. There is an interest in using deep learning methods to obviate the need for physicists to manually develop such features. Benchmark results using Bayesian Decision Trees from a standard physics package and 5-layer neural networks are presented in the original paper. The last 500,000 examples are used as a test set.

## Attribute Information:

The first column is the class label (1 for signal, 0 for background), followed by the 28 features (21 low-level features then 7 high-level features): lepton pT, lepton eta, lepton phi, missing energy magnitude, missing energy phi, jet 1 pt, jet 1 eta, jet 1 phi, jet 1 b-tag, jet 2 pt, jet 2 eta, jet 2 phi, jet 2 b-tag, jet 3 pt, jet 3 eta, jet 3 phi, jet 3 b-tag, jet 4 pt, jet 4 eta, jet 4 phi, jet 4 b-tag, m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb. For more detailed information about each feature see the original paper.

# Divide into train and test parts

In [9]:
seed = 42
train_data, test_data = data.randomSplit([0.8, 0.2], seed)
train_data.write.mode('overwrite').format('noop').save()
test_data.write.mode('overwrite').format('noop').save()

data.unpersist()

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string]

# AutoML params

In [10]:
roles = {
    "target": "_c0"
}
task = SparkTask("binary")
use_algos = [["lgb"]]
cv = 2

# Fitting and prediction

In [11]:
automl = SparkTabularAutoML(
    spark=spark,
    task=task,
    general_params={"use_algos": use_algos},
    lgb_params={'use_single_dataset_mode': True },
    reader_params={"cv": cv, "advanced_roles": False}
)

oof_predictions = automl.fit_predict(
    train_data,
    roles=roles
)

/home/user/projects/LightAutoML/lightautoml/spark/ml_algo/boost_lgbm.py:398: RuntimeWarning: Maximum validation size for SparkBoostLGBM is exceeded: 401067 > 10000. Reducing validation size down to maximum.
  warnings.warn(f"Maximum validation size for SparkBoostLGBM is exceeded: {valid_size} > {max_val_size}. "


[LightGBM] [Info] Number of positive: 211715, number of negative: 187586
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0,318274 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6132
[LightGBM] [Info] Number of data points in the train set: 399301, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0,530214 -> initscore=0,121004
[LightGBM] [Info] Start training from score 0,121004


/home/user/projects/LightAutoML/lightautoml/spark/utils.py:191: RuntimeWarning: Attempting to calculate shape on not cached dataframe. It may take too much time.
  warnings.warn("Attempting to calculate shape on not cached dataframe. "
/home/user/projects/LightAutoML/lightautoml/spark/ml_algo/boost_lgbm.py:398: RuntimeWarning: Maximum validation size for SparkBoostLGBM is exceeded: 401067 > 10000. Reducing validation size down to maximum.
  warnings.warn(f"Maximum validation size for SparkBoostLGBM is exceeded: {valid_size} > {max_val_size}. "


[LightGBM] [Info] Number of positive: 211715, number of negative: 187586
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0,038219 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6132
[LightGBM] [Info] Number of data points in the train set: 399301, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0,530214 -> initscore=0,121004
[LightGBM] [Info] Start training from score 0,121004


/home/user/projects/LightAutoML/lightautoml/spark/ml_algo/boost_lgbm.py:398: RuntimeWarning: Maximum validation size for SparkBoostLGBM is exceeded: 399301 > 10000. Reducing validation size down to maximum.
  warnings.warn(f"Maximum validation size for SparkBoostLGBM is exceeded: {valid_size} > {max_val_size}. "


[LightGBM] [Info] Number of positive: 212539, number of negative: 188528
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0,033555 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6132
[LightGBM] [Info] Number of data points in the train set: 401067, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0,529934 -> initscore=0,119879
[LightGBM] [Info] Start training from score 0,119879


# Score calculation

In [12]:
score = task.get_dataset_metric()
metric_value = score(oof_predictions)

te_pred = automl.predict(test_data, add_reader_attrs=True)
score = task.get_dataset_metric()
test_metric_value = score(te_pred)

print(f"OOF score: {metric_value}")
print(f"TEST score: {test_metric_value}")

OOF score: 0.8247978240800539
TEST score: 0.8291288777512692
